In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
data = pd.read_csv('50_Startups.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


In [4]:
data.State.unique()

array(['New York', 'California', 'Florida'], dtype=object)

In [5]:
data.describe()

R&D Spend  Administration  Marketing Spend         Profit
count      50.000000       50.000000        50.000000      50.000000
mean    73721.615600   121344.639600    211025.097800  112012.639200
std     45902.256482    28017.802755    122290.310726   40306.180338
min         0.000000    51283.140000         0.000000   14681.400000
25%     39936.370000   103730.875000    129300.132500   90138.902500
50%     73051.080000   122699.795000    212716.240000  107978.190000
75%    101602.800000   144842.180000    299469.085000  139765.977500
max    165349.200000   182645.560000    471784.100000  192261.830000

In [6]:
#Features and label
features = data.iloc[:,:-1].values
label = data.iloc[:,[-1]].values

In [7]:
#Deal with categorical data
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
stateOHE = ohe.fit_transform(features[:,3].reshape(-1,1))
finalFeatures = np.concatenate((stateOHE,features[:,[0,1,2]]), axis=1 )

In [8]:
#Train test split

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(finalFeatures, label, test_size=0.2, random_state=10)

In [9]:
print(len(X_train))

40


In [10]:
from math import sqrt
sampleSize = int(round(sqrt(len(X_train))))
print(sampleSize)

6


In [11]:
# Model using Linear Reg algorithm

from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression


algorithm = LinearRegression()
modelBag = BaggingRegressor(n_estimators=sampleSize, #No of weak learners
                         base_estimator=algorithm) #The algo to be used for learning


In [12]:
modelBag.fit(X_train, y_train.reshape(len(y_train),))

BaggingRegressor(base_estimator=LinearRegression(), n_estimators=6)

In [13]:
#Quaality
# SL = 0.08
# CL = 0.92

print(modelBag.score(X_train,y_train))
print(modelBag.score(X_test,y_test))

0.9364622988414638
0.989062326141131


In [15]:
# Testing for Generalization
SL = 0.03
CL = 0.97

from sklearn.model_selection import train_test_split

algorithm = LinearRegression()
for randomState in range(1,101):
    X_train,X_test,y_train,y_test = train_test_split(finalFeatures,label,test_size=0.2,random_state=randomState)
    modelBag = BaggingRegressor(n_estimators=sampleSize, base_estimator=algorithm)
    modelBag.fit(X_train,y_train.reshape(len(y_train),))
    trainScore = modelBag.score(X_train,y_train)
    testScore = modelBag.score(X_test,y_test)
    if testScore > trainScore and testScore >= CL:
        print("Test {} Train {} RS {}".format(testScore,trainScore,randomState))

Test 0.9821991640952369 Train 0.9372270787775709 RS 2
Test 0.9841200526421795 Train 0.9363645991695011 RS 10
Test 0.9720289197102819 Train 0.9405795333343264 RS 14
Test 0.9801391747010397 Train 0.9414738870201421 RS 22
Test 0.9809565967303331 Train 0.930260693693702 RS 45
Test 0.9773679289934779 Train 0.9384895869447719 RS 53
Test 0.9827683529665505 Train 0.9379009939599816 RS 69
Test 0.9742254798581653 Train 0.9382668690025899 RS 85


In [17]:
# Model using Decision Tree Reg

from sklearn.tree import DecisionTreeRegressor

In [23]:
algorithm = DecisionTreeRegressor()
X_train,X_test,y_train,y_test = train_test_split(finalFeatures,label,test_size=0.2,random_state=51)

for iteration in range(1,101):
    modelBag = BaggingRegressor(n_estimators=sampleSize, base_estimator=algorithm)
    modelBag.fit(X_train,y_train.reshape(len(y_train),))
    trainScore = modelBag.score(X_train,y_train)
    testScore = modelBag.score(X_test,y_test)
    if testScore > trainScore and testScore >= CL:
        print("Test {} Train {} ITR {}".format(testScore,trainScore,iteration))

Test 0.9702924622888913 Train 0.9612671130697198 ITR 29
Test 0.9824191568268805 Train 0.9752829218847554 ITR 39
Test 0.9825858215512705 Train 0.9704757353596041 ITR 44
Test 0.9708708522095373 Train 0.9561661277326697 ITR 45
Test 0.9713036790314526 Train 0.968184640078367 ITR 79
